# Convert Ising Model to HUBO

The following ariable state mapping of $1 \to 0, -1 \to 1$ from spin to binary variables is performed where $s_i \in \{-1, 1\}, x_i \in \{0, 1 \}$:

$$
\begin{align*}
s_i = 1 - 2x_i \equiv [1, - 2x_i]
\end{align*}
$$


$$
\begin{align*}
s_i \times s_j \times s_k &= [1, - 2x_i] \times [1, - 2x_j] \times [1, - 2x_k]
\end{align*}
$$

In [1]:
from collections import defaultdict
import ast
import pickle
from Cplex_HuboSolver import CPLEX_HUBOsolver
import warnings
warnings.simplefilter('ignore')

# Load instance
https://github.com/lanl/QAOA_vs_QA/tree/main/problem_instances

In [2]:
instance_dir = "../instances/SpinGlass"
instance_name = "ibm_washington_5"
with open(f"{instance_dir}/{instance_name}.txt", "r") as f:
    file = f.read()
instance = ast.literal_eval(file)

# Convert Ising to HUBO

In [3]:
def convert_ising_to_hubo(terms, hubo=None):
    if hubo is None:
        hubo = defaultdict(int)
    for vars, coeff in terms.items():
        if isinstance(vars, int):
            vars = tuple([vars])
        order = len(vars)
        # s_i = 1 - 2x_i -> [1, -2x_i]
        # s_i * s_j * s_k = [1, -2x_i] * [1, -2x_j] * [1, -2x_k]
        hubo[-1] += coeff
        for i in range(1, 2**order):
            bin_str = format(i, f"0{order}b")
            coeff_hubo = coeff
            vars_hubo = []
            for j, b in enumerate(bin_str):
                if b == "1":
                    coeff_hubo *= -2
                    vars_hubo.append(vars[j])
            hubo[tuple(sorted(vars_hubo))] += coeff_hubo
    return hubo

In [4]:
hubo = defaultdict(int)
sum_terms = 0
for i in range(len(instance)):
    for j in range(len(instance[i])):
        terms = instance[i][j]
        print(f"order = {i} \t number = {j} \t num_terms  = {len(terms)}")
        sum_terms += len(terms)
        hubo = convert_ising_to_hubo(terms, hubo)
print(f"sum_terms = {sum_terms}")
# hubo terms would be larger than original terms due to higher order terms.
print(f"sum_terms_hubo = {len(hubo)} (including constant term)")

order = 0 	 number = 0 	 num_terms  = 73
order = 0 	 number = 1 	 num_terms  = 54
order = 1 	 number = 0 	 num_terms  = 54
order = 1 	 number = 1 	 num_terms  = 47
order = 1 	 number = 2 	 num_terms  = 41
order = 2 	 number = 0 	 num_terms  = 69
sum_terms = 338
sum_terms_hubo = 408 (including constant term)


In [5]:
hubo

defaultdict(int,
            {-1: -32,
             (1,): 4,
             (3,): -4,
             (5,): 0,
             (7,): 4,
             (9,): 0,
             (11,): 4,
             (13,): 0,
             (14,): 4,
             (15,): 0,
             (16,): 4,
             (17,): 0,
             (19,): 8,
             (21,): 4,
             (23,): 4,
             (25,): 4,
             (27,): -4,
             (29,): -8,
             (31,): 0,
             (33,): 8,
             (34,): 0,
             (35,): 4,
             (36,): 0,
             (38,): -8,
             (40,): 4,
             (42,): 8,
             (44,): 4,
             (46,): 0,
             (48,): 0,
             (50,): -8,
             (52,): 4,
             (53,): 0,
             (54,): 0,
             (55,): 0,
             (57,): 4,
             (59,): 0,
             (61,): 4,
             (63,): -4,
             (65,): -4,
             (67,): 0,
             (69,): 8,
             (71,): 4,
             (72

In [6]:
with open(f"{instance_dir}/{instance_name}_hubo.pickle", "wb") as f:
    pickle.dump(hubo, f)

## Batch

In [7]:
terms

{(1, 0, 2): -1,
 (14, 0, 18): -1,
 (3, 2, 4): -1,
 (5, 4, 6): 1,
 (15, 4, 22): 1,
 (7, 6, 8): 1,
 (16, 8, 26): -1,
 (11, 10, 12): -1,
 (17, 12, 30): -1,
 (19, 18, 20): -1,
 (21, 20, 22): -1,
 (33, 20, 39): -1,
 (23, 22, 24): -1,
 (25, 24, 26): -1,
 (34, 24, 43): -1,
 (27, 26, 28): 1,
 (29, 28, 30): 1,
 (35, 28, 47): -1,
 (31, 30, 32): -1,
 (36, 32, 51): 1,
 (38, 37, 39): 1,
 (52, 37, 56): -1,
 (40, 39, 41): -1,
 (42, 41, 43): -1,
 (53, 41, 60): -1,
 (44, 43, 45): -1,
 (46, 45, 47): 1,
 (54, 45, 64): 1,
 (48, 47, 49): -1,
 (50, 49, 51): 1,
 (55, 49, 68): -1,
 (57, 56, 58): -1,
 (59, 58, 60): 1,
 (71, 58, 77): -1,
 (61, 60, 62): 1,
 (63, 62, 64): 1,
 (72, 62, 81): 1,
 (65, 64, 66): -1,
 (67, 66, 68): 1,
 (73, 66, 85): -1,
 (69, 68, 70): -1,
 (74, 70, 89): 1,
 (76, 75, 77): -1,
 (90, 75, 94): -1,
 (78, 77, 79): -1,
 (80, 79, 81): 1,
 (91, 79, 98): 1,
 (82, 81, 83): 1,
 (84, 83, 85): -1,
 (92, 83, 102): -1,
 (86, 85, 87): -1,
 (88, 87, 89): -1,
 (93, 87, 106): 1,
 (95, 94, 96): 1,
 (97, 96

In [8]:
instance_list = ["ibm_geneva_0", "ibmq_guadalupe_0"]

for instance_name in instance_list:
    with open(f"{instance_dir}/{instance_name}.txt", "r") as f:
        file = f.read()
    instance = ast.literal_eval(file)
    print(instance_name)
    hubo = defaultdict(int)
    sum_terms = 0
    qubits_set = set()
    for i in range(len(instance)):
        for j in range(len(instance[i])):
            terms = instance[i][j]
            print(f"\torder = {i} \t number = {j} \t num_terms  = {len(terms)}")
            sum_terms += len(terms)
            hubo = convert_ising_to_hubo(terms, hubo)
            for q in terms.keys():
                if type(q) == int:
                    qubits_set.add(q)
                else:
                    for qq in q:
                        qubits_set.add(qq)
    print(f"\tnum_qubits = {len(qubits_set)}")
    print(f"\tsum_terms = {sum_terms}")
    # hubo terms would be larger than original terms due to higher order terms.
    print(f"\tsum_terms_hubo = {len(hubo)} (including constant term)")
    with open(f"{instance_dir}/{instance_name}_hubo.pickle", "wb") as f:
        pickle.dump(hubo, f)

ibm_geneva_0
	order = 0 	 number = 0 	 num_terms  = 17
	order = 0 	 number = 1 	 num_terms  = 10
	order = 1 	 number = 0 	 num_terms  = 10
	order = 1 	 number = 1 	 num_terms  = 9
	order = 1 	 number = 2 	 num_terms  = 9
	order = 2 	 number = 0 	 num_terms  = 11
	num_qubits = 27
	sum_terms = 66
	sum_terms_hubo = 78 (including constant term)
ibmq_guadalupe_0
	order = 0 	 number = 0 	 num_terms  = 10
	order = 0 	 number = 1 	 num_terms  = 6
	order = 1 	 number = 0 	 num_terms  = 6
	order = 1 	 number = 1 	 num_terms  = 6
	order = 1 	 number = 2 	 num_terms  = 4
	order = 2 	 number = 0 	 num_terms  = 6
	num_qubits = 16
	sum_terms = 38
	sum_terms_hubo = 45 (including constant term)


# Load HUBO data

In [9]:
instance_dir = "../instances/SpinGlass"
instance_name = "ibm_washington_5"
with open(f"{instance_dir}/{instance_name}_hubo.pickle", "rb") as f:
  data = pickle.load(f)
  print(data)

defaultdict(<class 'int'>, {-1: -32, (1,): 4, (3,): -4, (5,): 0, (7,): 4, (9,): 0, (11,): 4, (13,): 0, (14,): 4, (15,): 0, (16,): 4, (17,): 0, (19,): 8, (21,): 4, (23,): 4, (25,): 4, (27,): -4, (29,): -8, (31,): 0, (33,): 8, (34,): 0, (35,): 4, (36,): 0, (38,): -8, (40,): 4, (42,): 8, (44,): 4, (46,): 0, (48,): 0, (50,): -8, (52,): 4, (53,): 0, (54,): 0, (55,): 0, (57,): 4, (59,): 0, (61,): 4, (63,): -4, (65,): -4, (67,): 0, (69,): 8, (71,): 4, (72,): 4, (73,): 4, (74,): 0, (76,): 4, (78,): 0, (80,): 0, (82,): 0, (84,): 0, (86,): 4, (88,): 0, (90,): 0, (91,): -8, (92,): 0, (93,): -8, (95,): 0, (97,): 0, (99,): 4, (101,): -4, (103,): -4, (105,): 0, (107,): 4, (109,): 4, (110,): 4, (111,): -4, (112,): -4, (113,): 4, (115,): 4, (117,): 0, (119,): 8, (121,): 0, (123,): -4, (125,): -4, (0,): 10, (2,): 2, (4,): -2, (6,): 2, (8,): 6, (10,): 4, (12,): 0, (18,): 2, (20,): 10, (22,): 10, (24,): 14, (26,): 2, (28,): -6, (30,): 2, (32,): 2, (37,): -2, (39,): 6, (41,): 10, (43,): 10, (45,): 6, (47,

In [10]:
hubo_terms = {}
offset = 0
qubits_set = set()
ord = -1
for key, value in data.items():
    if key == -1:
        offset = value
        continue
    hubo_terms[key] = value
    ord = max(ord, len(key))
    for i in key:
        qubits_set.add(i)
N = len(qubits_set)
print(f"num_qubits = {N} \t order = {ord}")
hubo_data = [N, hubo_terms]

num_qubits = 127 	 order = 3


In [11]:
cplex_result = CPLEX_HUBOsolver().solve(hubo_data)

print(
    '-'*80,
    '[CPLEX Result]',
    f'   optimal value: {cplex_result.fval}',
    f'   optimal solution: {cplex_result.x}',
    f'   status: {cplex_result.status.name}',
    '-'*80,
    sep='\n'
)

print(f"optimal value = {cplex_result.fval + offset}")

--------------------------------------------------------------------------------
[CPLEX Result]
   optimal value: -166.0
   optimal solution: [0 0 0 1 1 1 1 0 0 1 1 0 1 1 0 0 0 1 0 0 0 0 0 1 1 1 1 1 0 1 0 1 1 0 1 1 0
 0 1 0 0 0 0 0 1 1 1 1 0 1 1 1 0 1 1 0 0 0 1 0 1 0 0 1 1 0 1 1 1 1 0 1 0 0
 0 1 1 0 0 0 0 1 1 1 1 0 0 1 0 1 0 1 0 0 1 0 1 1 0 0 1 0 1 0 0 1 1 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0]
   status: SUCCESS
--------------------------------------------------------------------------------
optimal value = -198.0
